# AtomsBase integration

[AtomsBase.jl](https://github.com/JuliaMolSim/AtomsBase.jl) is a common interface
for representing atomic structures in Julia. DFTK directly supports using such
structures to run a calculation as is demonstrated here.

In [1]:
using DFTK
using AtomsBuilder

## Feeding an AtomsBase AbstractSystem to DFTK

In this example we construct a bulk silicon system using the `bulk` function
from [AtomsBuilder](https://github.com/JuliaMolSim/AtomsBuilder.jl). This function
uses tabulated data to set up a reasonable starting geometry and lattice for bulk silicon.

In [2]:
system = bulk(:Si)

FlexibleSystem(Si₂, periodicity = TTT):
    cell_vectors      : [       0    2.715    2.715;
                            2.715        0    2.715;
                            2.715    2.715        0]u"Å"

    Atom(Si, [       0,        0,        0]u"Å")
    Atom(Si, [  1.3575,   1.3575,   1.3575]u"Å")


By default the atoms of an `AbstractSystem` employ the bare Coulomb potential.
To employ pseudpotential models (which is almost always advisable for
plane-wave DFT) one employs the `pseudopotential` keyword argument in
model constructors such as `model_DFT`.
For example we can employ a `PseudoFamily` object
from the [PseudoPotentialData](https://github.com/JuliaMolSim/PseudoPotentialData.jl)
package. See its documentation for more information on the available
pseudopotential families and how to select them.

In [3]:
using PseudoPotentialData  # defines PseudoFamily

pd_lda_family = PseudoFamily("dojo.nc.sr.lda.v0_4_1.oncvpsp3.standard.upf")
model = model_DFT(system;
                  functionals=LDA(),
                  temperature=1e-3,
                  pseudopotentials=pd_lda_family)

Model(lda_x+lda_c_pw, 3D):
    lattice (in Bohr)    : [0         , 5.13061   , 5.13061   ]
                           [5.13061   , 0         , 5.13061   ]
                           [5.13061   , 5.13061   , 0         ]
    unit cell volume     : 270.11 Bohr³

    atoms                : Si₂
    atom potentials      : ElementPsp(Si, "/home/runner/.julia/artifacts/1ea71a84cf375286564538a9cab789991f4bf1f4/Si.upf")
                           ElementPsp(Si, "/home/runner/.julia/artifacts/1ea71a84cf375286564538a9cab789991f4bf1f4/Si.upf")

    num. electrons       : 8
    spin polarization    : none
    temperature          : 0.001 Ha
    smearing             : DFTK.Smearing.FermiDirac()

    terms                : Kinetic()
                           AtomicLocal()
                           AtomicNonlocal()
                           Ewald(nothing)
                           PspCorrection()
                           Hartree()
                           Xc(lda_x, lda_c_pw)
                   

Alternatively the `pseudopotentials` object also accepts a `Dict{Symbol,String}`,
which provides for each element symbol the filename or identifier
of the pseudopotential to be employed, e.g.

In [4]:
model = model_DFT(system;
                  functionals=LDA(),
                  temperature=1e-3,
                  pseudopotentials=Dict(:Si => "hgh/lda/si-q4"))

Model(lda_x+lda_c_pw, 3D):
    lattice (in Bohr)    : [0         , 5.13061   , 5.13061   ]
                           [5.13061   , 0         , 5.13061   ]
                           [5.13061   , 5.13061   , 0         ]
    unit cell volume     : 270.11 Bohr³

    atoms                : Si₂
    atom potentials      : ElementPsp(Si, "hgh/lda/si-q4")
                           ElementPsp(Si, "hgh/lda/si-q4")

    num. electrons       : 8
    spin polarization    : none
    temperature          : 0.001 Ha
    smearing             : DFTK.Smearing.FermiDirac()

    terms                : Kinetic()
                           AtomicLocal()
                           AtomicNonlocal()
                           Ewald(nothing)
                           PspCorrection()
                           Hartree()
                           Xc(lda_x, lda_c_pw)
                           Entropy()

We can then discretise such a model and solve:

In [5]:
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.921728421539                   -0.69    6.1    811ms
  2   -7.926162546313       -2.35       -1.22    1.0    844ms
  3   -7.926838364745       -3.17       -2.37    1.9    164ms
  4   -7.926861192904       -4.64       -3.04    2.1    177ms
  5   -7.926861651753       -6.34       -3.40    2.0    165ms
  6   -7.926861670428       -7.73       -3.81    1.5    150ms
  7   -7.926861678435       -8.10       -4.06    1.5    149ms
  8   -7.926861681791       -8.47       -4.95    1.4    143ms
  9   -7.926861681853      -10.21       -5.14    2.6    170ms
 10   -7.926861681871      -10.73       -6.22    1.0    135ms
 11   -7.926861681873      -11.87       -6.40    3.0    190ms
 12   -7.926861681873   +  -13.97       -6.18    1.1    149ms
 13   -7.926861681873      -13.80       -7.52    1.0    143ms
 14   -7.926861681873      -15.05       -7.50    3.5    649ms
 15   -7.

If we did not want to use
[AtomsBuilder](https://github.com/JuliaMolSim/AtomsBuilder.jl)
we could of course use any other package
which yields an AbstractSystem object. This includes:

### Reading a system using AtomsIO

Read a file using [AtomsIO](https://github.com/mfherbst/AtomsIO.jl),
which directly yields an AbstractSystem.

In [6]:
using AtomsIO
system = load_system("Si.extxyz");

Run the LDA calculation:

In [7]:
pseudopotentials = Dict(:Si => "hgh/lda/si-q4")
model  = model_DFT(system; pseudopotentials, functionals=LDA(), temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.921726398856                   -0.69    6.0    270ms
  2   -7.926165093708       -2.35       -1.22    1.0    140ms
  3   -7.926838786201       -3.17       -2.37    1.9    187ms
  4   -7.926861237945       -4.65       -3.02    2.1    190ms
  5   -7.926861644067       -6.39       -3.36    1.9    159ms
  6   -7.926861666876       -7.64       -3.72    1.6    141ms
  7   -7.926861679375       -7.90       -4.15    1.1    136ms
  8   -7.926861681805       -8.61       -5.17    1.6    149ms
  9   -7.926861681864      -10.23       -5.25    3.1    188ms
 10   -7.926861681872      -11.09       -6.45    1.0    149ms
 11   -7.926861681873      -12.36       -6.70    3.1    186ms
 12   -7.926861681873      -14.45       -7.52    1.0    137ms
 13   -7.926861681873      -14.75       -7.85    2.8    173ms
 14   -7.926861681873   +    -Inf       -9.61    1.2    143ms


The same could be achieved using [ExtXYZ](https://github.com/libAtoms/ExtXYZ.jl)
by `system = Atoms(read_frame("Si.extxyz"))`,
since the `ExtXYZ.Atoms` object is directly AtomsBase-compatible.

### Directly setting up a system in AtomsBase

In [8]:
using AtomsBase
using Unitful
using UnitfulAtomic

# Construct a system in the AtomsBase world
a = 10.26u"bohr"  # Silicon lattice constant
lattice = a / 2 * [[0, 1, 1.],  # Lattice as vector of vectors
                   [1, 0, 1.],
                   [1, 1, 0.]]
atoms  = [:Si => ones(3)/8, :Si => -ones(3)/8]
system = periodic_system(atoms, lattice; fractional=true)

# Now run the LDA calculation:
pseudopotentials = Dict(:Si => "hgh/lda/si-q4")
model  = model_DFT(system; pseudopotentials, functionals=LDA(), temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.921699249201                   -0.69    5.9    297ms
  2   -7.926169614530       -2.35       -1.22    1.0    160ms
  3   -7.926842051907       -3.17       -2.37    1.9    171ms
  4   -7.926864622968       -4.65       -3.02    2.2    193ms
  5   -7.926865053845       -6.37       -3.36    1.9    163ms
  6   -7.926865077319       -7.63       -3.72    1.4    138ms
  7   -7.926865090285       -7.89       -4.13    1.2    136ms


## Obtaining an AbstractSystem from DFTK data

At any point we can also get back the DFTK model as an
AtomsBase-compatible `AbstractSystem`:

In [9]:
second_system = atomic_system(model)

FlexibleSystem(Si₂, periodicity = TTT):
    cell_vectors      : [       0     5.13     5.13;
                             5.13        0     5.13;
                             5.13     5.13        0]u"a₀"

    Atom(Si, [  1.2825,   1.2825,   1.2825]u"a₀")
    Atom(Si, [ -1.2825,  -1.2825,  -1.2825]u"a₀")


Similarly DFTK offers a method to the `atomic_system` and `periodic_system` functions
(from AtomsBase), which enable a seamless conversion of the usual data structures for
setting up DFTK calculations into an `AbstractSystem`:

In [10]:
lattice = 5.431u"Å" / 2 * [[0 1 1.];
                           [1 0 1.];
                           [1 1 0.]];
Si = ElementPsp(:Si, load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

third_system = atomic_system(lattice, atoms, positions)

FlexibleSystem(Si₂, periodicity = TTT):
    cell_vectors      : [       0  5.13155  5.13155;
                          5.13155        0  5.13155;
                          5.13155  5.13155        0]u"a₀"

    Atom(Si, [ 1.28289,  1.28289,  1.28289]u"a₀")
    Atom(Si, [-1.28289, -1.28289, -1.28289]u"a₀")
